RECURSIVE

In [ ]:
import fitz  # PyMuPDF
import os
import re

def clean_text(text):
    """
    Membersihkan dan merapikan teks hasil ekstraksi.
    """
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = re.sub(r'\n+', ' ', text)
    return text.strip()

def load_pdf_text(pdf_path):
    """
    Load dan ekstrak teks dari PDF.
    """
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            page_text = page.get_text("text")
            page_text = clean_text(page_text)
            full_text += page_text + " "
        
        return clean_text(full_text)
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return ""

def recursive_chunk(text, chunk_size, separators=None):
    """
    Membagi teks secara rekursif menjadi chunk dengan panjang maksimum menggunakan separator yang ditentukan.
    """
    if not text.strip():
        return []

    if separators is None:
        separators = ["."]

    for separator in separators:
        if separator in text[:chunk_size]:
            split_index = text[:chunk_size].rfind(separator) + len(separator)
            chunk = text[:split_index]
            remaining_text = text[split_index:].strip()
            return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)

    chunk = text[:chunk_size]
    remaining_text = text[chunk_size:].strip()
    return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)

def save_chunks_to_file(chunks, output_file):
    """
    Menyimpan semua chunk ke dalam satu file.
    """
    with open(output_file, "w", encoding="utf-8") as f:
        for idx, chunk in enumerate(chunks, 1):
            f.write(f"Chunk {idx:03d}:\n")
            f.write(chunk)
            f.write("\n\n")
    
    print(f"Semua chunks telah disimpan ke dalam file: {output_file}")

def main():
    # Konfigurasi
    pdf_path = "./data/tko.pdf"
    output_file = "./output_chunks/output_recursive.txt"  # File untuk menyimpan hasil chunking
    chunk_size = 1000

    # Load PDF dan ekstrak teks
    print("Mengekstrak teks dari PDF...")
    full_text = load_pdf_text(pdf_path)

    if not full_text:
        print("Tidak ada teks yang bisa diekstrak. Program dihentikan.")
        return

    # Chunk teks secara rekursif
    print("Membagi teks menjadi chunks secara rekursif...")
    chunks = recursive_chunk(full_text, chunk_size)

    print(f"Selesai membagi menjadi {len(chunks)} chunks.")

    # Simpan semua chunk ke satu file
    save_chunks_to_file(chunks, output_file)

if __name__ == "__main__":
    main()

Mengekstrak teks dari PDF...
Membagi teks menjadi chunks secara rekursif...
Selesai membagi menjadi 11 chunks.
Semua chunks telah disimpan dalam folder: ./output_chunks/recursive


OVERLAP

In [ ]:
import os
import fitz
import nltk
import re
import textwrap
from langchain.text_splitter import RecursiveCharacterTextSplitter
from nltk.tokenize import sent_tokenize

class PDFChunker:
    def __init__(self, chunk_size=1000, chunk_overlap=200, line_width=80):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.line_width = line_width
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF file with better formatting."""
        try:
            doc = fitz.open(pdf_path)
            text_blocks = []
            
            for page in doc:
                blocks = page.get_text("blocks")
                for block in blocks:
                    clean_text = block[4].strip()
                    if clean_text:
                        text_blocks.append(clean_text)
            
            text = " ".join(text_blocks)
            return self.clean_text(text)
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return None
        finally:
            if 'doc' in locals():
                doc.close()

    def clean_text(self, text):
        """Clean and normalize text while preserving structure."""
        if not text:
            return ""
        
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'([.!?])\s*', r'\1 ', text)
        text = re.sub(r'\s+([.!?])', r'\1', text)
        text = re.sub(r'\n\s*\n', '\n', text)
        
        return text.strip()

    def create_improved_chunks(self, text):
        """Create chunks with improved sentence handling."""
        if not text:
            return []

        sentences = sent_tokenize(text)
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence = sentence.strip()
            sentence_length = len(sentence)

            if not current_chunk:
                current_chunk.append(sentence)
                current_length = sentence_length
                continue

            if current_length + len(" ") + sentence_length <= self.chunk_size:
                current_chunk.append(sentence)
                current_length += len(" ") + sentence_length
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_length

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        final_chunks = []
        for i in range(len(chunks)):
            if i > 0:
                prev_chunk_sentences = sent_tokenize(chunks[i-1])
                overlap_sentences = prev_chunk_sentences[-2:] if len(prev_chunk_sentences) > 2 else prev_chunk_sentences
                current_chunk = " ".join(overlap_sentences) + " " + chunks[i]
                final_chunks.append(current_chunk)
            else:
                final_chunks.append(chunks[i])

        return final_chunks

    def wrap_text(self, text):
        """Wrap text to specified line width while preserving paragraphs."""
        # Split text into paragraphs
        paragraphs = text.split('\n\n')
        
        # Wrap each paragraph
        wrapped_paragraphs = []
        for paragraph in paragraphs:
            # Wrap the paragraph text
            wrapped = textwrap.fill(paragraph.strip(), width=self.line_width)
            wrapped_paragraphs.append(wrapped)
        
        # Join paragraphs with double newlines
        return '\n\n'.join(wrapped_paragraphs)

    def save_chunks_to_file(self, chunks, output_folder="chunks_output"):
        """Save chunks to a single file with proper text wrapping."""
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        output_file = os.path.join(output_folder, "output_overlap.txt")
        
        try:
            with open(output_file, "w", encoding="utf-8") as file:
                for i, chunk in enumerate(chunks, 1):
                    # Write chunk header
                    header = f"Chunk {i}"
                    file.write(f"{header}\n")
                    file.write("="* len(header) + "\n\n")
                    
                    # Write wrapped chunk content
                    wrapped_content = self.wrap_text(chunk)
                    file.write(wrapped_content)
                    file.write("\n\n\n")  # Add extra spacing between chunks

            print(f"✅ {len(chunks)} chunks successfully saved to '{output_file}'")
        except Exception as e:
            print(f"Error saving chunks: {e}")

    def process_pdf(self, pdf_path):
        """Process PDF and create better chunks."""
        text = self.extract_text_from_pdf(pdf_path)
        if not text:
            return []
        
        return self.create_improved_chunks(text)

    def analyze_chunks(self, chunks):
        """Analyze chunks with improved metrics."""
        if not chunks:
            return {
                "total_chunks": 0,
                "average_size": 0,
                "size_stats": {},
                "sentence_stats": {}
            }

        chunk_sizes = [len(chunk) for chunk in chunks]
        sentences_per_chunk = [len(sent_tokenize(chunk)) for chunk in chunks]

        analysis = {
            "total_chunks": len(chunks),
            "average_size": sum(chunk_sizes) / len(chunks),
            "size_stats": {
                "min": min(chunk_sizes),
                "max": max(chunk_sizes),
                "avg": sum(chunk_sizes) / len(chunks)
            },
            "sentence_stats": {
                "min_sentences": min(sentences_per_chunk),
                "max_sentences": max(sentences_per_chunk),
                "avg_sentences": sum(sentences_per_chunk) / len(sentences_per_chunk)
            }
        }
        return analysis

def main():
    # Initialize chunker with line width for text wrapping
    chunker = PDFChunker(chunk_size=1000, chunk_overlap=200, line_width=80)
    
    # Configuration
    pdf_path = "./data/tko.pdf"
    output_folder = "./output_chunks/"
    
    # Process PDF
    chunks = chunker.process_pdf(pdf_path)
    
    if chunks:
        # Save results
        chunker.save_chunks_to_file(chunks, output_folder)
        
        # Analyze and display results
        analysis = chunker.analyze_chunks(chunks)
        print("\nChunking Analysis:")
        print(f"Total chunks: {analysis['total_chunks']}")
        print(f"Average chunk size: {analysis['size_stats']['avg']:.0f} characters")
        print(f"Min/Max chunk size: {analysis['size_stats']['min']}/{analysis['size_stats']['max']} characters")
        print(f"\nSentences per chunk:")
        print(f"Min: {analysis['sentence_stats']['min_sentences']:.0f}")
        print(f"Max: {analysis['sentence_stats']['max_sentences']:.0f}")
        print(f"Average: {analysis['sentence_stats']['avg_sentences']:.1f}")

if __name__ == "__main__":
    main()

✅ Chunk 1 saved: chunk_001.txt
✅ Chunk 2 saved: chunk_002.txt
✅ Chunk 3 saved: chunk_003.txt
✅ Chunk 4 saved: chunk_004.txt
✅ Chunk 5 saved: chunk_005.txt
✅ Chunk 6 saved: chunk_006.txt
✅ Chunk 7 saved: chunk_007.txt
✅ Chunk 8 saved: chunk_008.txt
✅ Chunk 9 saved: chunk_009.txt
✅ Chunk 10 saved: chunk_010.txt

✅ 10 chunks successfully processed and saved in './output_chunks/overlap'.


TOKEN BASE

In [ ]:
import os
import tiktoken
import fitz  # PyMuPDF
from datetime import datetime
from typing import List, Tuple
import re
import logging
import gc
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Set up logging
logging.basicConfig(
    filename='pdf_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class SentenceChunkProcessor:
    def __init__(self, chunk_size=500):
        self.chunk_size = chunk_size
        self.encoder = tiktoken.get_encoding("cl100k_base")

    def read_pdf_and_clean(self, file_path: str) -> str:
        logging.info(f"Started processing PDF: {file_path}")
        doc = fitz.open(file_path)
        text = " ".join([page.get_text("text") for page in doc])
        text = re.sub(r'\s+', ' ', text).strip()

        # Menghindari pemisahan angka yang salah (4. 3.)
        text = re.sub(r'(\d+)\.\s(?=\d+)', r'\1|', text)

        logging.info(f"Successfully read and cleaned PDF: {file_path}")
        return text

    def restore_numbering(self, text: str) -> str:
        """Mengembalikan angka yang sebelumnya diganti (4|3 -> 4. 3.)"""
        return text.replace('|', '. ')

    def chunk_text_by_sentences(self, text: str) -> List[Tuple[str, int]]:
        text = self.restore_numbering(text)  # Kembalikan format angka
        sentences = sent_tokenize(text)
        chunks = []
        current_tokens = []
        current_text = ""

        for sentence in sentences:
            sentence_tokens = self.encoder.encode(sentence)
            
            if len(current_tokens) + len(sentence_tokens) > self.chunk_size:
                if current_tokens:
                    chunks.append((self.encoder.decode(current_tokens), len(current_tokens)))
                    current_tokens = []  # Reset tanpa overlap
                    current_text = ""
            
            current_tokens.extend(sentence_tokens)
            current_text += sentence
            
            gc.collect()
        
        if current_tokens:
            chunks.append((self.encoder.decode(current_tokens), len(current_tokens)))

        logging.info(f"Processed and chunked text into {len(chunks)} chunks")
        return chunks

    def save_chunks(self, content_chunks: List[Tuple[str, int]], output_path: str) -> None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"{os.path.splitext(output_path)[0]}_{timestamp}.txt"

        try:
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(f"Processing Date: {datetime.now()}\n")
                f.write(f"Chunk Size: {self.chunk_size} tokens\n")
                f.write("=" * 50 + "\n\n")

                for i, (chunk, token_count) in enumerate(content_chunks, 1):
                    f.write(f"=== Chunk {i} ===\n")
                    f.write(f"Tokens: {token_count}\n")
                    f.write(chunk + "\n\n")
                    f.write("-" * 30 + "\n\n")

            logging.info(f"Successfully saved chunks to: {output_path}")
            logging.info(f"Total chunks: {len(content_chunks)}")
        except Exception as e:
            logging.error(f"Error saving chunks: {str(e)}")
            raise

# Main Program
if __name__ == "__main__":
    file_path = "./data/tko.pdf"
    output_path = "./output_chunks/output_token_base.txt"

    processor = SentenceChunkProcessor(chunk_size=500)

    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' tidak ditemukan!")
    else:
        try:
            print("🔍 Memulai proses membaca PDF...")
            text = processor.read_pdf_and_clean(file_path)
            print(f"Teks PDF terbaca (50 karakter pertama): {text[:50]}...")

            print("🔍 Memulai proses chunking...")
            content_chunks = processor.chunk_text_by_sentences(text)
            print(f"Total chunk yang dibuat: {len(content_chunks)}")

            print("🔍 Menyimpan hasil chunking ke output file...")
            processor.save_chunks(content_chunks, output_path)
            print(f"✅ File output tersimpan di: {output_path}")

        except Exception as e:
            print(f"❌ Terjadi error: {str(e)}")

[nltk_data] Downloading package punkt to C:\Users\Agus
[nltk_data]     Syuhada\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


🔍 Memulai proses membaca PDF...
Teks PDF terbaca (50 karakter pertama): TATA KERJA ORGANISASI PENGELOLAAN AKUN APLIKASI No...
🔍 Memulai proses chunking...
Total chunk yang dibuat: 6
🔍 Menyimpan hasil chunking ke folder output...
✅ Semua chunk tersimpan di folder: ./output_chunks/token_base


CONTEXT ENRICHED

In [14]:
import os
import re
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

class AdvancedChunking:
    def __init__(self, ukuran_chunk=1000, overlap=200):
        self.ukuran_chunk = ukuran_chunk
        self.overlap = overlap
        self.output_folder = 'output_chunks'
        os.makedirs(self.output_folder, exist_ok=True)

    def baca_pdf(self, path_file):
        """Membaca teks dari file PDF."""
        pdf_reader = PdfReader(path_file)
        teks = "".join([halaman.extract_text() or "" for halaman in pdf_reader.pages])
        return teks

    def preprocessing_teks(self, teks):
        """Membersihkan teks dari spasi berlebih dan karakter tidak perlu."""
        return re.sub(r'\s+', ' ', teks).strip()

    def chunk_dengan_konteks(self, teks):
        """Membagi teks menjadi chunk dengan mempertahankan sedikit konteks dari chunk sebelumnya dan berikutnya."""
        teks_bersih = self.preprocessing_teks(teks)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.ukuran_chunk, chunk_overlap=self.overlap, length_function=len,
            separators=[".", "\n\n", "\n", " ", ""]
        )

        chunks = text_splitter.create_documents([teks_bersih])
        chunks_dengan_konteks = []

        for i, chunk in enumerate(chunks):
            konteks_sebelum = chunks[i - 1].page_content if i > 0 else ""
            konteks_sesudah = chunks[i + 1].page_content if i < len(chunks) - 1 else ""

            chunk_konteks = f"{konteks_sebelum} {chunk.page_content} {konteks_sesudah}".strip()
            chunk_konteks = chunk_konteks.rstrip('.') + '.'

            metadata = {
                'chunk_id': i,
                'total_chunks': len(chunks),
                'panjang_chunk': len(chunk_konteks),
                'konteks_sebelum': konteks_sebelum[:50],
                'konteks_sesudah': konteks_sesudah[:50]
            }

            chunks_dengan_konteks.append({'konten': chunk_konteks, 'metadata': metadata})

        return chunks_dengan_konteks

    def proses_folder_pdf(self, folder_path):
        """Memproses semua file PDF dalam folder dan membaginya menjadi chunk."""
        hasil = {}
        total_chunk = 0

        for filename in os.listdir(folder_path):
            if filename.endswith('.pdf'):
                file_path = os.path.join(folder_path, filename)
                print(f"\n{'='*50}")
                print(f"Memproses file: {filename}")
                print(f"{'='*50}")

                teks_pdf = self.baca_pdf(file_path)
                chunks = self.chunk_dengan_konteks(teks_pdf)

                total_chunk += len(chunks)
                hasil[filename] = {'chunks': chunks}

                # Menyimpan hasil chunking ke file teks
                chunk_file = os.path.join(self.output_folder, f"{filename}_chunks.txt")
                with open(chunk_file, "w", encoding="utf-8") as f:
                    for i, chunk in enumerate(chunks, 1):
                        f.write(f"Chunk {i}:\n{chunk['konten']}\n\n")

                print(f"Hasil chunking disimpan di: {chunk_file}")

        return hasil

if __name__ == "__main__":
    chunker = AdvancedChunking(ukuran_chunk=1000, overlap=200)
    FOLDER_PATH = "./data"

    try:
        hasil_chunking = chunker.proses_folder_pdf(FOLDER_PATH)
    except Exception as e:
        print(f"Terjadi error: {str(e)}")


Memproses file: dokumen_pdf.pdf
Hasil chunking disimpan di: output_chunks\dokumen_pdf.pdf_chunks.txt

Memproses file: tko.pdf
Hasil chunking disimpan di: output_chunks\tko.pdf_chunks.txt


SEMANTIC

In [15]:
import fitz  # PyMuPDF
import re

def read_pdf(file_path):
    """Membaca teks dari file PDF."""
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

# def clean_text(text):
#     """Membersihkan teks dari karakter khusus, spasi berlebih, dan format yang tidak diperlukan."""
#     text = re.sub(r'\n+', '\n', text)  # Menghapus newline berlebih
#     text = re.sub(r'[^a-zA-Z0-9.,\n ]+', '', text)  # Menghapus karakter khusus
#     text = re.sub(r' +', ' ', text)  # Menghapus spasi berlebih
#     return text.strip()

def clean_text(text):
    """Membersihkan teks dari karakter khusus, spasi berlebih, dan format yang tidak diperlukan."""
    text = text.replace("\n", " ").strip() 
    text = " ".join(text.split())  
    return text

def chunk_text(text, chunk_size=1000):
    """Membagi teks menjadi chunk lebih kecil berdasarkan ukuran karakter."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

def save_to_txt(filename, chunks):
    """Menyimpan hasil ke file .txt."""
    with open(filename, "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            f.write(f"Chunk {i+1}:\n{chunk}\n\n")

# Path ke file PDF
pdf_path = "./data/tko.pdf"
output_txt = "./output_chunks/output_semantic.txt"

# Membaca, membersihkan, dan membagi teks
raw_text = read_pdf(pdf_path)
cleaned_text = clean_text(raw_text)
chunks = chunk_text(cleaned_text)

# Menyimpan output ke file .txt
save_to_txt(output_txt, chunks)

print(f"Hasil telah disimpan dalam {output_txt}")

Hasil telah disimpan dalam ./output_chunks/output_semantic.txt


ADVANCE SEMANTIC

In [16]:
import os
import spacy
import docx
import fitz
import numpy as np
import gensim
from gensim import corpora
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama.llms import OllamaLLM

# Load NLP model
nlp = spacy.load("en_core_web_sm")
OLLAMA_MODEL = "llama3.2"
llm = OllamaLLM(model=OLLAMA_MODEL)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def read_txt(file_path):
    print(f"Reading TXT file: {file_path}")
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def read_pdf(file_path):
    print(f"Reading PDF file: {file_path}")
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

def read_docx(file_path):
    print(f"Reading DOCX file: {file_path}")
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def clean_text(text):
    print("Preprocessing text: Cleaning and normalizing...")
    text = text.replace("\n", " ").strip() 
    text = " ".join(text.split())  
    return text

def semantic_chunking(text, chunk_size=1000):
    print("Performing semantic chunking...")
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    
    if not sentences:
        return []
    
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    if chunk:
        chunks.append(chunk.strip())
    
    print(f"Generated {len(chunks)} chunks.")
    return chunks

def extract_topics(text, num_topics=5):
    print("Extracting topics using LDA...")
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]
    dictionary = corpora.Dictionary([words])
    corpus = [dictionary.doc2bow(words)]
    lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=4)
    topics = lda_model.show_topics(formatted=False)
    
    return topics[0] if topics else None

def process_document(file_path, output_folder):
    print(f"Processing document: {file_path}")
    ext = file_path.split(".")[-1].lower()
    
    if ext == "txt":
        text = read_txt(file_path)
    elif ext == "pdf":
        text = read_pdf(file_path)
    elif ext == "docx":
        text = read_docx(file_path)
    else:
        print(f"Unsupported file format: {ext}")
        return
    
    cleaned_text = clean_text(text)
    chunks = semantic_chunking(cleaned_text)
    
    output_file = os.path.join(output_folder, os.path.basename(file_path).split(".")[0] + "_chunks.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            print(f"Processing chunk {i+1}...")
            best_topic = extract_topics(chunk)
            if best_topic:
                topic_words = ", ".join([word for word, _ in best_topic[1]])
                topic_str = f"Best Topic: {topic_words}"
            else:
                topic_str = "Best Topic: No topics found"
            f.write(f"--- Chunk {i+1} ---\n{chunk}\n{topic_str}\n\n")
    
    print(f"Finished processing: {file_path} -> {output_file}")

def main():
    input_folder = "./data"
    output_folder = "./output_chunks/"
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            process_document(file_path, output_folder)

if __name__ == "__main__":
    main()

Processing document: ./data\.gitkeep
Unsupported file format: gitkeep
Processing document: ./data\dokumen_pdf.pdf
Reading PDF file: ./data\dokumen_pdf.pdf
Preprocessing text: Cleaning and normalizing...
Performing semantic chunking...
Generated 5 chunks.
Processing chunk 1...
Extracting topics using LDA...
Processing chunk 2...
Extracting topics using LDA...
Processing chunk 3...
Extracting topics using LDA...
Processing chunk 4...
Extracting topics using LDA...
Processing chunk 5...
Extracting topics using LDA...
Finished processing: ./data\dokumen_pdf.pdf -> ./output_chunks/dokumen_pdf_chunks.txt
Processing document: ./data\tko.pdf
Reading PDF file: ./data\tko.pdf
Preprocessing text: Cleaning and normalizing...
Performing semantic chunking...
Generated 10 chunks.
Processing chunk 1...
Extracting topics using LDA...
Processing chunk 2...
Extracting topics using LDA...
Processing chunk 3...
Extracting topics using LDA...
Processing chunk 4...
Extracting topics using LDA...
Processing c

AGENTIC

In [17]:
import os
import json
import re
from IPython.display import Markdown
from phi.agent import Agent
from phi.model.ollama import Ollama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from utils.document_processor import DocumentProcessor  

# Inisialisasi path dan model
DATA_PATH = "./data"
INDEX_PATH = "faiss_index"
CHUNKED_DATA_PATH = "./output_chunks"  
METADATA_PATH = "./metadata"  
OLLAMA_MODEL = "llama3.2"

# Pastikan folder tersedia
os.makedirs(CHUNKED_DATA_PATH, exist_ok=True)
os.makedirs(METADATA_PATH, exist_ok=True)

# Inisialisasi model Ollama
llm = Ollama(id=OLLAMA_MODEL)

# Inisialisasi embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Inisialisasi document processor
docs = DocumentProcessor()

# Inisialisasi Agent dengan model Ollama
agent = Agent(model=llm, show_tool_calls=True, markdown=True)

# Parameter chunking
CHUNK_SIZE = 1200  # Ditingkatkan untuk efisiensi
MIN_CHUNK_SIZE = 500  # Gabungkan chunk kecil
MAX_CHUNKS = 30  # Batasi jumlah chunk

extracted_docs = []

def clean_text(text):
    """ Membersihkan teks dari karakter kosong, whitespace berlebih, dan simbol aneh """
    text = re.sub(r'\s+', ' ', text).strip() 
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  
    return text

def clean_agent_output(text):
    """ Membersihkan output dari agent agar tidak mengandung pemisah yang tidak perlu """
    text = re.sub(r'\n?###.*?\n', '\n', text)  # Hapus judul markdown seperti ### Section
    text = re.sub(r'\n?\*\*\*.*?\n', '\n', text)  # Hapus pemisah ***
    text = re.sub(r'\n?-{3,}\n?', '\n', text)  # Hapus garis pemisah ---
    text = re.sub(r'(\s*-{2,}\s*)', ' ', text)  # Hapus pemisah --
    text = re.sub(r'(\s*\*{2,}\s*)', ' ', text)  # Hapus ** pemisah tebal
    text = re.sub(r'(\s*\*\s*)', ' ', text)  # Hapus * pemisah tunggal
    text = re.sub(r'(\s*-\s*)', ' ', text)  # Hapus pemisah -
    text = re.sub(r'\n{2,}', '\n\n', text).strip() 
    return text

for filename in os.listdir(DATA_PATH):
    valid_extensions = ('.pdf', '.docx', '.txt')
    if not filename.lower().endswith(valid_extensions):
        continue

    filepath = os.path.join(DATA_PATH, filename)

    try:
        with open(filepath, "rb") as f:
            document = f.read()
            result = docs.process_document(document, filename)

        if not result or len(result) < 4:
            print(f"[WARNING] Gagal memproses {filename}, melewati file ini.")
            continue

        plain_text = clean_text(result[3])  
        print(f"[INFO] {filename} - Panjang teks sebelum pemrosesan: {len(plain_text)}")

        # Gunakan agent untuk chunking secara bertahap
        structured_text = ""
        start_idx = 0
        chunk_count = 0

        while start_idx < len(plain_text) and chunk_count < MAX_CHUNKS:
            chunk_text = plain_text[start_idx:start_idx + CHUNK_SIZE]
            response = agent.run(
                f"Split the following text into meaningful segments ensuring logical separation:\n{chunk_text}",
                max_tokens=8000
            )
            
            if isinstance(response, str):
                structured_text += clean_agent_output(response) + "\n\n"
            elif isinstance(response, dict):
                structured_text += clean_agent_output(response.get("text", "")) + "\n\n"
            else:
                structured_text += clean_agent_output(getattr(response, "content", str(response))) + "\n\n"
            
            start_idx += CHUNK_SIZE
            chunk_count += 1

        structured_text = structured_text.strip()
        chunked_texts = structured_text.split("\n\n")

        # Gabungkan chunk yang terlalu pendek
        optimized_chunks = []
        temp_chunk = ""

        for chunk in chunked_texts:
            chunk = chunk.strip()
            if len(chunk) < MIN_CHUNK_SIZE:
                temp_chunk += " " + chunk
            else:
                if temp_chunk:
                    optimized_chunks.append(temp_chunk.strip())
                    temp_chunk = ""
                optimized_chunks.append(chunk)

        if temp_chunk:
            optimized_chunks.append(temp_chunk.strip())

        chunk_data = [{"chunk_id": i+1, "text": chunk.strip()} 
                      for i, chunk in enumerate(optimized_chunks[:MAX_CHUNKS]) if chunk.strip()]

        metadata = {
            "filename": filename,
            "total_chunks": len(chunk_data),
            "total_length": len(plain_text)
        }

        extracted_docs.extend([
            Document(page_content=chunk["text"], metadata={"chunk_id": chunk["chunk_id"], **metadata}) 
            for chunk in chunk_data
        ])

        # Simpan hasil chunking ke file
        chunked_filepath = os.path.join(CHUNKED_DATA_PATH, f"chunked_{filename}.txt")
        with open(chunked_filepath, "w", encoding="utf-8") as chunked_file:
            for chunk in chunk_data:
                chunked_file.write(f"Chunk {chunk['chunk_id']}:\n")
                chunked_file.write(f"{chunk['text']}\n")
                chunked_file.write("\n---\n\n")  

        metadata_filepath = os.path.join(METADATA_PATH, f"metadata_{filename}.json")
        with open(metadata_filepath, "w", encoding="utf-8") as metadata_file:
            json.dump(metadata, metadata_file, indent=4)

        print(f"[INFO] Total chunks generated for {filename}: {len(chunk_data)}")

    except Exception as e:
        print(f"[ERROR] Error processing {filename}: {e}")

# Simpan ke FAISS hanya jika ada dokumen yang diproses
if extracted_docs:
    vector_store = FAISS.from_documents(extracted_docs, embedding_model)
    vector_store.save_local(INDEX_PATH)
    print("[SUCCESS] Proses chunking selesai. Hasilnya disimpan dalam 'chunked_data' dan metadata di 'metadata'.")
else:
    print("[INFO] Tidak ada dokumen yang berhasil diproses.")

[INFO] dokumen_pdf.pdf - Panjang teks sebelum pemrosesan: 3843
[INFO] Total chunks generated for dokumen_pdf.pdf: 5
[INFO] tko.pdf - Panjang teks sebelum pemrosesan: 8838
[INFO] Total chunks generated for tko.pdf: 10
[SUCCESS] Proses chunking selesai. Hasilnya disimpan dalam 'chunked_data' dan metadata di 'metadata'.


MODEL

In [ ]:
import os
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

OLLAMA_MODEL = "llama3.2"

# Load LLM model
model = OllamaLLM(model=OLLAMA_MODEL)

# Template prompt untuk ringkasan
template = """
1. Apa tujuan dari tata kerja organisasi?
2. Apa pengertian Onsite Support dan Aplikasi Upstream?
3. Apa dokumen dan referensi terkait Tata Kerja Organisasi?
4. Bagaimana prosedur permintaan baru akun aplikasi?
Gunakan hanya informasi yang terdapat dalam dokumen.  

Dokumen:
"{document}"
"""

prompt = ChatPromptTemplate.from_template(template)

def summarize_text(text):
    """Meringkas teks menggunakan model LLM"""
    chain = prompt | model
    response = chain.invoke({"document": text})
    return response

def read_all_chunks(folder):
    """Membaca semua file chunk dalam folder dan menggabungkan isinya"""
    all_text = []
    
    # Ambil semua file dalam folder dan urutkan berdasarkan nama file
    files = sorted([f for f in os.listdir(folder) if f.startswith("chunk_") and f.endswith(".txt")])
    
    for file in files:
        file_path = os.path.join(folder, file)
        with open(file_path, "r", encoding="utf-8") as f:
            all_text.append(f.read())  # Tambahkan isi file ke list
    
    return "\n\n".join(all_text)  # Gabungkan semua isi file dengan pemisah newline

# Baca semua chunk dari folder
output_folder = "output_chunks/recursive"
document_text = read_all_chunks(output_folder)

# Buat ringkasan
summary = summarize_text(document_text)

# Tampilkan hasil sebagai Markdown
Markdown(summary)

Based on the provided text, I'll answer your questions:

1. Apa tujuan dari tata kerja organisasi?

The purpose of organizational workflow is to manage and control the processes within an organization. In this case, it seems that the main goal is to ensure proper management of user access and account creation in an IT system.

2. Apa apakah yang dimaksud dengan User Access Review (UAR)?

From the text, it appears that UAR refers to a review process for new account requests or updates to existing accounts. This process involves various roles, such as Application Programmer, Technical Writer, Information Steward, and Business Analyst, working together to ensure that user access is properly managed.

In summary:

* The organizational workflow aims to manage and control user access in an IT system.
* User Access Review (UAR) is a process for reviewing new account requests or updates to existing accounts, involving multiple roles.